<a href="https://colab.research.google.com/github/Saranya32113/FMML_LAB_1/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%


The accuracy of a machine learning model on the validation set can be affected when you change the percentage of data allocated to the validation set. Here's what typically happens:

Increasing Validation Set Percentage:

Pros: Increasing the percentage of data in the validation set can provide a more robust estimate of your model's performance. It ensures that a larger portion of your data is used for validation, which can help detect overfitting or generalization issues early.
Cons: However, if you allocate too much data to the validation set, you may have fewer samples for training, which can lead to underfitting. Your model might not learn enough from the reduced training data, resulting in lower overall accuracy.
Reducing Validation Set Percentage:

Pros: Reducing the percentage of data in the validation set means you have more data available for training. This can help your model learn better and potentially achieve higher training accuracy.
Cons: On the downside, with a smaller validation set, you might not have a reliable estimate of how well your model generalizes to new, unseen data. This could lead to overfitting, where the model performs well on the training data but poorly on new data.
The choice of the validation set percentage should strike a balance between having enough data to evaluate the model effectively and having sufficient data for the model to learn from. It often depends on the size of your dataset, the complexity of your model, and the specific problem you're solving. Cross-validation techniques can also be useful to mitigate the impact of varying validation set sizes.


The accuracy of a machine learning model on the validation set can be affected when you change the percentage of data allocated to the validation set. Here's what typically happens:

Increasing Validation Set Percentage:

Pros: Increasing the percentage of data in the validation set can provide a more robust estimate of your model's performance. It ensures that a larger portion of your data is used for validation, which can help detect overfitting or generalization issues early.
Cons: However, if you allocate too much data to the validation set, you may have fewer samples for training, which can lead to underfitting. Your model might not learn enough from the reduced training data, resulting in lower overall accuracy.
Reducing Validation Set Percentage:

Pros: Reducing the percentage of data in the validation set means you have more data available for training. This can help your model learn better and potentially achieve higher training accuracy.
Cons: On the downside, with a smaller validation set, you might not have a reliable estimate of how well your model generalizes to new, unseen data. This could lead to overfitting, where the model performs well on the training data but poorly on new data.
The choice of the validation set percentage should strike a balance between having enough data to evaluate the model effectively and having sufficient data for the model to learn from. It often depends on the size of your dataset, the complexity of your model, and the specific problem you're solving. Cross-validation techniques can also be useful to mitigate the impact of varying validation set sizes.

In practice, it's common to use a split like 80% for training and 20% for validation, but this can vary based on your specific needs and the amount of data available.



Larger Training Set:

Pros: A larger training set generally allows your model to learn more about the underlying patterns in the data. This can result in a model that generalizes better to unseen data (including the test set).
Cons: If your training set is very large relative to the validation set, the validation set may not be representative enough to reliably estimate model performance. This can lead to an overly optimistic or pessimistic view of the model's capabilities.
Larger Validation Set:

Pros: A larger validation set provides a more robust estimate of your model's performance. It means you are evaluating the model on a broader sample of your data, reducing the likelihood of random variations affecting your evaluation.
Cons: If the validation set is too large compared to the training set, you might have limited data for training, which can lead to underfitting. Your model may not learn enough to perform well on the test set.
Balanced Split:

Pros: A balanced split, where you allocate an appropriate portion of data to both training and validation sets, strikes a good balance. It allows your model to learn while providing a representative estimate of its performance.
Cons: The trade-off is that you may have slightly less data for both training and validation compared to the cases of having a significantly larger set in one or the other

The percentage to reserve for the validation set can vary depending on the size of your dataset, the complexity of your model, and the specific problem you're trying to solve. However, a commonly used split is the 80-20 rule, where you allocate 80% of your data for training and 20% for validation.

This split often strikes a reasonable balance between having enough data for training (to prevent underfitting) and having a representative validation set (to get a reliable estimate of model performance). It's a good starting point for many machine learning tasks, especially when you have a moderate-sized dataset.

That said, there is no one-size-fits-all answer. In some cases, you might use a 70-30 or 90-10 split, depending on your specific needs. For very large datasets, you can allocate a smaller percentage to validation, while for smaller datasets, you might use a larger validation set.

Additionally, techniques like k-fold cross-validation can be employed to further balance the trade-off between training and validation sizes. In k-fold cross-validation, the data is divided into k subsets, and the model is trained and validated k times, with each subset serving as the validation set once.

Ultimately, the choice of the validation set percentage should be based on empirical testing and consideration of the characteristics of your data and problem domain. You may need to experiment with different split ratios to find the one that works best for your specific use case.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.
Yes, averaging validation accuracy across multiple splits, such as in k-fold cross-validation, can provide more consistent and reliable results compared to a single validation split. It helps reduce the impact of random variability in the data distribution or initial split. By taking the average, you get a more robust estimate of your model's performance, which can be particularly useful for assessing how well your model generalizes to unseen data.

2.

Averaging validation accuracy across multiple splits, as in k-fold cross-validation, provides a more accurate estimate of a model's performance on unseen data compared to a single validation split. However, it's important to note that the validation accuracy obtained through cross-validation is still an estimate of the model's performance on unseen test data.

The purpose of cross-validation is to provide a more robust and less biased estimate of model performance by using multiple validation sets. While it can give you a better idea of how your model is likely to perform on unseen data, the ultimate measure of a model's accuracy on test data comes from a separate, held-out test set that the model has never seen during training or validation. This final test set is crucial for assessing the true generalization performance of your model.

3.
The number of iterations (or epochs) in the training of a machine learning model can have an impact on the estimate of its performance, but it doesn't necessarily mean that a higher number of iterations always leads to a better estimate.

Here's how the number of iterations can affect the estimate:

1. Underfitting: If you have too few iterations, the model may not have enough time to learn from the data, leading to underfitting. In such cases, the validation accuracy may be low, and increasing the number of iterations can improve the estimate to some extent.

2. Convergence: There's a point at which the model converges, meaning it has learned as much as it can from the available data. Beyond this point, increasing the number of iterations might not significantly improve the estimate of the model's performance on unseen data.

3. Overfitting: On the other hand, if you have too many iterations, the model might start overfitting the training data, meaning it learns the noise in the data rather than the underlying patterns. This can lead to high validation accuracy but poor generalization to unseen data.

Finding the right number of iterations typically involves a trade-off. You want to train your model long enough to capture the underlying patterns in the data without overfitting. This balance can be determined through techniques like early stopping, where you monitor validation performance and stop training when it starts to degrade.

In summary, a higher number of iterations is not guaranteed to provide a better estimate of model performance. The choice of the number of iterations should be based on a balance between underfitting and overfitting, and it's often determined through experimentation and monitoring validation performance.

The number of iterations (epochs) in training a machine learning model can impact the estimate of its performance, but it doesn't necessarily mean that a higher number of iterations will always result in a better estimate.

Here's how the number of iterations can affect the estimate:

1. **Underfitting**: If you have too few iterations, the model may not have sufficient training to capture the underlying patterns in the data. This can lead to underfitting, where the model's performance on both the training and validation sets is subpar. In this case, increasing the number of iterations can potentially improve the estimate by allowing the model to learn more from the data.

2. **Convergence**: There is a point at which the model reaches convergence, meaning it has learned as much as it can from the training data. Once a model has converged, further iterations may not significantly improve the estimate of its performance on unseen data. Adding more iterations beyond this point could lead to overfitting.

3. **Overfitting**: On the contrary, if you have too many iterations, the model may start to memorize the training data, including noise and outliers. This can result in overfitting, where the model performs very well on the training data but poorly on the validation or test data. In this case, more iterations do not lead to a better estimate of the model's generalization performance.

In practice, the number of iterations should be chosen carefully, often through techniques like early stopping. Early stopping monitors the model's performance on a validation set and stops training when it starts to degrade, which helps prevent overfitting. The ideal number of iterations depends on factors like the complexity of the model, the size and quality of the training data, and the presence of regularization techniques.

So, it's not always the case that more iterations result in a better estimate of a model's performance. The key is to strike a balance between underfitting and overfitting to achieve the best generalization to unseen data.